Import

In [0]:
import pyspark.sql.functions as F

1. Leia e visualize os dois datasets (dica: ambos estão em formato csv)

In [0]:
high_temps = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema" , "true")\
                .load("/databricks-datasets/weather/high_temps")

low_temps = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema" , "true")\
                .load("/databricks-datasets/weather/low_temps")

2. Conte o número de linhas de ambos.

In [0]:
print(f"DataSet high_temps: \n\nTotal Linhas >--- {high_temps.count()} \n\nDados: \n")

high_temps.limit(5).display()

DataSet high_temps: 

Total Linhas >--- 1369 

Dados: 



date,temp
2015-01-01T00:00:00.000+0000,42
2015-01-02T00:00:00.000+0000,42
2015-01-03T00:00:00.000+0000,41
2015-01-04T00:00:00.000+0000,51
2015-01-05T00:00:00.000+0000,54


In [0]:
print(f"DataSet low_temps: \n\nTotal linhas >--- {low_temps.count()} \n\nDados: \n")

low_temps.limit(5).display()

DataSet low_temps: 

Total linhas >--- 1369 

Dados: 



date,temp
2015-01-01T00:00:00.000+0000,26
2015-01-02T00:00:00.000+0000,32
2015-01-03T00:00:00.000+0000,35
2015-01-04T00:00:00.000+0000,38
2015-01-05T00:00:00.000+0000,49


3. Verifique se há valores repetidos na coluna "date" de ambos os datasets.

In [0]:
df_high_temps = high_temps.dropDuplicates(["date"])
df_low_temps = low_temps.dropDuplicates(["date"])

high_count = df_high_temps.count()
low_count = df_low_temps.count()

if high_count < 1369:
    print(f'DataFrame high_temps possuia {high_count - 1369} valores repetidos na coluna date, linhas removidas.\n')
else:
    print('DataFrame high_temps não possui valores repetidos na coluna date.\n')

if low_count < 1369:
    print(f'\nDataFrame low_temps possuia {low_count - 1369} valores repetidos na coluna date, linhas removidas.')
else:
    print('\nDataFrame low_temps não possui valores repetidos na coluna date.')

DataFrame high_temps não possui valores repetidos na coluna date.


DataFrame low_temps não possui valores repetidos na coluna date.


4. Verifique se há valores nulos nos datasets.

In [0]:
df_high_temps.dropna()
df_low_temps.dropna()

high_count = df_high_temps.count()
low_count = df_low_temps.count()

if high_count < 1369:
    print(f'DataFrame high_temps possuia {high_count - 1369} valores nulos, linhas removidas.\n')
else:
    print('DataFrame high_temps não possui valores nulos.\n')

if low_count < 1369:
    print(f'\nDataFrame low_temps possuia {low_count - 1369} valores nulos,, linhas removidas.')
else:
    print('\nDataFrame low_temps não possui valores nulos.')

DataFrame high_temps não possui valores nulos.


DataFrame low_temps não possui valores nulos.


5. Verifique as datas mínima e máxima de cada dataset.

In [0]:
min_max_high = df_high_temps.select("date") \
                            .agg(F.min('date'), 
                                 F.max('date'))

min_max_low = df_low_temps.select("date") \
                          .agg(F.min('date'), 
                               F.max('date'))

print("df_high_temps: ")
display(min_max_high)

print("="*65)

print("\ndf_low_temps: ")
display(min_max_low)


df_high_temps: 


min(date),max(date)
2015-01-01T00:00:00.000+0000,2018-09-30T00:00:00.000+0000



df_low_temps: 


min(date),max(date)
2015-01-01T00:00:00.000+0000,2018-09-30T00:00:00.000+0000


6. Verifique o schema dos datasets.

In [0]:
print("df_high_temps: ")

df_high_temps.printSchema()

print("="*65)
print("\ndf_low_temps: ")

df_low_temps.printSchema()

df_high_temps: 
root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)


df_low_temps: 
root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)



7. Faça um JOIN das duas tabelas, de forma que tenhamos apenas um dataset com as colunas "date", "higher_temp" e "lower_temp".

In [0]:
df_high_temps = df_high_temps.withColumnRenamed("temp", "higher_temp" )
                             
df_low_temps = df_low_temps.withColumnRenamed("date", "dateLow") \
                           .withColumnRenamed("temp", "lower_temp")

df_high_and_low_temps = df_high_temps.join(df_low_temps, df_high_temps.date == df_low_temps.dateLow, "inner") \
                                     .select("date", "higher_temp", "lower_temp")

8. Verifique o número de linhas da nova tabela e se há algum valor nulo (dica: o número de linhas deve ser idêntico ao número de linhas das tabelas separadas)

In [0]:
count_df = df_high_and_low_temps.count()
df_high_and_low_temps.dropna()

if count_df < 1369:
    print(f'DataFrame high_temps possuia {count_df - 1369} valores nulos, linhas removidas.\nPossui {count_df} linhas')
else:
    print(f'DataFrame high_temps não possui valores nulos. \nPossui {count_df} linhas')

DataFrame high_temps não possui valores nulos. 
Possui 1369 linhas


9. Crie uma coluna chamada "amplitude", que será higher_temp - lower_temp

In [0]:
df_temps= df_high_and_low_temps.withColumn("amplitude", F.col("higher_temp") - F.col("lower_temp"))

10. Encontre os dias com a maior e a menor amplitude de temperatura.

In [0]:
min_ampli = df_temps.agg(F.min('amplitude')) \
                    .head()[0]

max_ampli = df_temps.agg(F.max('amplitude')) \
                    .head()[0]


print("Dias com menor amplitude: ")

display(df_temps.select("*") \
                .filter(df_temps.amplitude == min_ampli))
print("="*65)
print("Dias com maior amplitude: ")

display(df_temps.select("*") \
                .filter(df_temps.amplitude == max_ampli))

Dias com menor amplitude: 


date,higher_temp,lower_temp,amplitude
2017-11-16T00:00:00.000+0000,44,42,2


Dias com maior amplitude: 


date,higher_temp,lower_temp,amplitude
2016-04-18T00:00:00.000+0000,89,54,35


11. Crie uma coluna chamada "mean", que será a média da temperatura máxima e mínima de cada dia.

In [0]:
df_temps = df_temps.withColumn("mean", (((F.col("higher_temp") + F.col("lower_temp")) / 2)))

12. Encontre os dias com a maior e a menor média de temperatura.

In [0]:
min_mean = df_temps.agg(F.min('mean')) \
                    .head()[0]

max_mean = df_temps.agg(F.max('mean')) \
                    .head()[0]


print("Dias com menor média de temperatura: ")

display(df_temps.select("*") \
                .filter(df_temps.mean == min_mean))
print("="*65)
print("Dias com maior média de temperatura: ")

display(df_temps.select("*") \
                .filter(df_temps.mean == max_mean))

Dias com menor média de temperatura: 


date,higher_temp,lower_temp,amplitude,mean
2017-01-03T00:00:00.000+0000,33,21,12,27.0


Dias com maior média de temperatura: 


date,higher_temp,lower_temp,amplitude,mean
2016-08-19T00:00:00.000+0000,95,69,26,82.0


13. Encontre uma solução para particionarmos o dataset por ANO e MÊS.

In [0]:
df_temps = df_temps.select(("*"),
                   F.year("date").alias("year"),
                   F.month("date").alias("month"))

14. Salve o novo dataset em uma pasta de sua preferência, particionado por ANO e MÊS.

In [0]:
df_temps.write.partitionBy("year", "month").mode("overwrite").parquet("/FileStore/tables/df_temps.parquet")

15. Agrupe o dataset por ANO e MÊS e encontre os seguintes valores agregados:
- Temperatura máxima de cada mês
- Temperatura mínima de cada mês
- Temperatura média de cada mês
- Amplitude de cada mês

In [0]:
df_temps = spark.read.parquet("/FileStore/tables/df_temps.parquet/")

df_temps_groupby = df_temps.groupBy("year", "month") 

In [0]:
display(df_temps_groupby.agg(F.max("higher_temp").alias("Temperatura máxima de cada mês"),
                             F.min("lower_temp").alias("Temperatura mínima de cada mês"),
                             F.avg("mean").alias("Temperatura média de cada mês"),
                             F.mean("amplitude").alias("Amplitude média de cada mês"),
                             F.min("amplitude").alias("Amplitude minima de cada mês"), 
                             F.max("amplitude").alias("Amplitude máxima de cada mês"))
                        .orderBy("year", "month"))

year,month,Temperatura máxima de cada mês,Temperatura mínima de cada mês,Temperatura média de cada mês,Amplitude média de cada mês,Amplitude minima de cada mês,Amplitude máxima de cada mês
2015,1,63,26,45.064516129032256,10.451612903225806,3,18
2015,2,62,33,48.75,11.571428571428571,4,22
2015,3,69,31,50.516129032258064,14.709677419354838,8,24
2015,4,77,37,51.36666666666667,17.066666666666666,10,27
2015,5,82,43,59.12903225806452,17.806451612903224,4,31
2015,6,92,49,67.66666666666667,22.466666666666665,8,29
2015,7,95,54,71.24193548387096,22.677419354838708,8,33
2015,8,92,54,68.70967741935483,20.516129032258064,5,32
2015,9,81,45,60.5,16.066666666666666,5,23
2015,10,74,45,57.24193548387097,12.67741935483871,4,25
